### Make sure mongodb service is up and running
### _First time only_: Import entities and relations into database

Bash:
```bash
mongorestore --db lobbyradar --collection entities dumps/entities.bson
mongorestore --db lobbyradar --collection relations dumps/relations.bson
```

## Connect to database

In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint, pformat

client = MongoClient() # Default address localhost
db = client.lobbyradar

## The collections

### Numbers

In [3]:
# The database has two collections
print("Collections in database: " + ", ".join(db.collection_names(include_system_collections=False)))

print("Entities: %s documents" % db.entities.count())
print("Relations: %s documents" % db.relations.count())

Collections in database: entities, relations
Entities: 26380 documents
Relations: 32137 documents


### Simple queries

In [18]:
print("Distict entity types:" + pformat(db.entities.distinct("type")))

Distict entity types:[u'entity', u'person']


In [19]:
print("Distict relation types:" + pformat(db.relations.distinct("type")))

Distict relation types:[u'general', u'publication']


In [20]:
# Some commands
print("First Entity document in database:" + pformat(db.entities.find_one()) + "\n")
#entity_cursor = db.entities.find() # Would find all entities
#entity_cursor.next() # Iterator for resultset

First Entity document in database:{u'_id': ObjectId('54bd3c748b934da06340f4c1'),
 u'aliases': [u'DIE LINKE',
              u'Die Linke',
              u'Partei DIE LINKE',
              u'DIE LINKE.',
              u'Linkspartei',
              u'Linkspartei.PDS',
              u'PDS',
              u'WASG',
              u'Partei des Demokratischen Sozialismus',
              u'Arbeit & soziale Gerechtigkeit \u2013 Die Wahlalternative',
              u'Wahlalternative',
              u'Wahlalternative Arbeit und soziale Gerechtigkeit'],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 807000),
 u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
            u'desc': u'Partei',
            u'format': u'string',
            u'id': u'2b1adb60a31d37cf9cc0fdccb75149456a425095c6bf5e77abac117ed1a69d0f',
            u'key': u'partei',
            u'updated': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
            u'value': u'Die L

In [21]:
print("First relation document in database:" + pformat(db.relations.find_one()) + "\n")
#relation_cursor = db.entities.find()
#relation_cursor.next()

First relation document in database:{u'_id': ObjectId('54bd3c968b934da063413717'),
 u'created': datetime.datetime(2015, 1, 19, 17, 19, 18, 521000),
 u'data': [{u'desc': u'Verbindung',
            u'format': u'association',
            u'importer': u'created by lobbyliste importer',
            u'key': u'association',
            u'value': {u'position': u'Vorstand',
                       u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
                       u'type': u'executive'}}],
 u'entities': [ObjectId('54bd3c768b934da06340f4c5'),
               ObjectId('54bd3c768b934da06340f4c7')],
 u'importer': u'lobbyliste',
 u'tags': [],
 u'type': u'general',
 u'updated': datetime.datetime(2015, 1, 19, 17, 19, 18, 521000),
 u'weight': 1}



### Aggregations

In [22]:
from bson.son import SON

# Aggregation pipeline that groups by type (entity/person) and counts the sum of documents in each group in a field "counter"
# The second step sorts the groups by counter and id 
pipeline = [
    {"$group": {"_id": "$type", "counter": {"$sum": 1}}},   # First step
    {"$sort": SON([("counter", -1), ("_id", -1)])}          # Second step
]

# Output as list to get all values at once instead of a cursor
print("Entitiy types count:")
print(list(db.entities.aggregate(pipeline, cursor={})))
print("Relation types count:")
print(list(db.relations.aggregate(pipeline, cursor={})))

Entitiy types count:
[{u'_id': u'person', u'counter': 19828}, {u'_id': u'entity', u'counter': 6552}]
Relation types count:
[{u'_id': u'general', u'counter': 32136}, {u'_id': u'publication', u'counter': 1}]


In [23]:
# Aggregation mit Gruppierung der Dokumente aus der collection entities nach Feld "importer", zählen der Gruppen und aufsteigend danach sortieren
pipeline = [ {"$group": {"_id": "$importer", "count": {"$sum": 1}}} , {"$sort": {"count":1} }]
list(db.entities.aggregate(pipeline, cursor={}))

[{u'_id': u'parteien', u'count': 1},
 {u'_id': u'parteispenden14', u'count': 6},
 {u'_id': u'kabinette', u'count': 34},
 {u'_id': u'seitenwechsler', u'count': 214},
 {u'_id': u'thinktanks', u'count': 309},
 {u'_id': u'parteispenden13', u'count': 311},
 {u'_id': u'laender', u'count': 370},
 {u'_id': u'dax', u'count': 541},
 {u'_id': None, u'count': 1063},
 {u'_id': u'pr', u'count': 1767},
 {u'_id': u'bundestag', u'count': 2046},
 {u'_id': u'parteispenden', u'count': 2914},
 {u'_id': u'lobbyliste', u'count': 16804}]

## Queries

### Entitäten mit Attribut "importer": "parteispenden14"

In [29]:
query = {'type': 'entity', 'importer': 'parteispenden14'}
projection = {'_id': 0, 'name': 1}
for i in db.entities.find(query, projection):
     pprint(i)

print("%s Entitäten mit importer parteispenden14 gefunden" % db.entities.find(query, projection).count())

{u'name': u'Firma R & W Industriebeteiligungen GmbH'}
{u'name': u'R&W Industriebeteiligungen GmbH'}
{u'name': u'MLPD'}
3 Entitäten mit importer parteispenden14 gefunden


#### Entitäten mit "importer": "parteispenden14", deren Namen zwischen G und H beginnen 

In [41]:
list(db.entities.find( {'importer': 'parteispenden14', 'name': {'$gte': 'G', '$lte': 'H'} } ))

[{u'_id': ObjectId('550f0eabe073b81931ff68fe'),
  u'aliases': [],
  u'created': datetime.datetime(2015, 3, 22, 18, 49, 14, 193000),
  u'data': [{u'auto': True,
    u'created': datetime.datetime(2015, 3, 22, 18, 49, 14, 193000),
    u'desc': u'Quelle',
    u'format': u'link',
    u'id': u'ae3aece79528dc33c4839fe6590a7f04931dd2552583a1deda7c1235510dc3cf',
    u'key': u'source',
    u'updated': datetime.datetime(2015, 3, 22, 18, 49, 14, 193000),
    u'value': {u'remark': u'Bundestag Drs. 18/3614',
     u'url': u'http://dip.bundestag.de/btd/18/037/1803757.pdf'}},
   {u'auto': True,
    u'created': datetime.datetime(2015, 3, 22, 18, 49, 14, 193000),
    u'desc': u'Adresse',
    u'format': u'address',
    u'id': u'57562edd16541b49aecce26513d95e417b9a3fa3fdd2c92d962c800f9a18ad6a',
    u'key': u'address',
    u'updated': datetime.datetime(2015, 3, 22, 18, 49, 14, 193000),
    u'value': {u'addr': u'',
     u'city': u'Duisburg',
     u'country': u'Germany',
     u'name': u'Gabi Georgiou',
     u

#### Gruppierung der Importer

In [105]:
# Gruppierung nach importer und Zählen der Elemente der Gruppen. Dann sortieren
pipeline = [ {"$group": {"_id": "$importer", "count": {"$sum": 1}}} , {"$sort": {"count":1} }]
list(db.entities.aggregate(pipeline, cursor={}))

[{u'_id': u'parteien', u'count': 1},
 {u'_id': u'parteispenden14', u'count': 6},
 {u'_id': u'kabinette', u'count': 34},
 {u'_id': u'seitenwechsler', u'count': 214},
 {u'_id': u'thinktanks', u'count': 309},
 {u'_id': u'parteispenden13', u'count': 311},
 {u'_id': u'laender', u'count': 370},
 {u'_id': u'dax', u'count': 541},
 {u'_id': None, u'count': 1063},
 {u'_id': u'pr', u'count': 1767},
 {u'_id': u'bundestag', u'count': 2046},
 {u'_id': u'parteispenden', u'count': 2914},
 {u'_id': u'lobbyliste', u'count': 16804}]

### Name von Entitäten, deren Namen zwischen X-Z beginnen

In [42]:
query = {'name': {'$gte': 'X', '$lte': 'Z'} }                                                                                                                                                  
projection = {'_id':0, 'name':1}                                                                                                                                                               
for i in db.entities.find(query, projection):                                                                                                                                                  
    pprint(i)   

{u'name': u'XLAB-Stiftung zur F\xf6rderung der Naturwissenschaften'}
{u'name': u'XVIDA'}
{u'name': u'Xander Heijnen'}
{u'name': u'Xander Hijnen'}
{u'name': u'Xaver Fischer'}
{u'name': u'Xaver Jung'}
{u'name': u'Xaver Kroner'}
{u'name': u'Xaver Schmidt'}
{u'name': u'Xavier Feys'}
{u'name': u'Xavier Gonzalez'}
{u'name': u'Xavier Roth le Gentil'}
{u'name': u'XertifiX e.V.'}
{u'name': u'Xinhua News Agency'}
{u'name': u'Xu Rong Heinrich'}
{u'name': u'YARA GmbH & Co. KG'}
{u'name': u'YOC AG'}
{u'name': u'Yade L\xfctz'}
{u'name': u'Yahoo'}
{u'name': u'Yan C. Steinschen'}
{u'name': u'Yanina Terentyeva'}
{u'name': u'Yankee Candle'}
{u'name': u'Yanki P\xfcrs\xfcn'}
{u'name': u'Yasmin Fahimi'}
{u'name': u'Yasushi Yamawaki'}
{u'name': u'Yello Strom'}
{u'name': u'Yeom Tae-Young'}
{u'name': u'Yesim Eraslan'}
{u'name': u'Yilmaz Kahraman'}
{u'name': u'Yingli Green Energy International'}
{u'name': u'Yoram-Illy Ehrlich'}
{u'name': u'Yorck Hillegaart'}
{u'name': u'Yorck Jetter'}
{u'name': u'Yorck Otto'}


### Entitäten anhand von Tags finden

#### A representative

In [49]:
# Query on a array fields (here tags field)                                                                                                                                                    
query = {'tags': "representative"}                                                                                                                                                             
one_representative = db.entities.find_one(query)                                                                                                                                                                    
print("\"%s\" is a representative." % one_representative.get("name"))
one_representative.get("tags")

"Ralf Denda" is a representative.


[u'lobbyist', u'lobbyismus', u'representative']

#### A representative or guy in Medien

In [54]:
# $in operator : any of the provided values should match                                                                                                                                       
query = {'tags': {'$in':["representative", "Medien"]}}                                                                                                                                         
print("\"%s\" is either a representative or working in Medien" % db.entities.find_one(query).get("name"))
db.entities.find_one(query).get("tags")

"Allianz Deutscher Produzenten - Film & Fernsehen e.V." is either a representative or working in Medien


[u'lobbyorganisation', u'lobbyismus', u'Medien']

#### A lobbyist AND guy in Medien

In [55]:
# $all operator: all values must match
query = {'tags': {'$all':["lobbyismus", "Medien"]}}                                                                                                                                            
print("\"%s\" is tagged with lobbyismus and working in Medien" % db.entities.find_one(query).get("name"))
db.entities.find_one(query).get("tags")

"Allianz Deutscher Produzenten - Film & Fernsehen e.V." is tagged with lobbyismus and working in Medien


[u'lobbyorganisation', u'lobbyismus', u'Medien']

#### Count tags

In [58]:
# in compararion with                                                                                                                                                                          
print("Die Datenbank enthält %s Entitäten mit Tag Medien und %s Entitäten mit Tag lobbyismus" % (db.entities.find({'tags': "Medien"}).count(), db.entities.find({'tags': "lobbyismus"}).count()))

Die Datenbank enthält 41 Entitäten mit Tag Medien und 16998 Entitäten mit Tag lobbyismus


### Access inner documents and arrays

#### Get the first 5 other known aliases for "Die Linke"

In [104]:
# dot notation for querying inner documents or projection                                                                                                                                      
projection = { "_id":0, "aliases": {"$slice": 5} }   
db.entities.find_one({ "name": "Die Linke"}, projection).get("aliases")

[u'DIE LINKE',
 u'Die Linke',
 u'Partei DIE LINKE',
 u'DIE LINKE.',
 u'Linkspartei']

### Entpacken von Arrays mit $unwind

In [119]:
# Aggregation, die das "data"-Array einer Entität zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden dann nach importer gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
pipeline = [
    {"$unwind": "$data"},
    {"$group": {"_id": "$importer", "count": {"$sum": 1}}},
    {"$sort": {"count":1} }
]
list(db.entities.aggregate(pipeline))

[{u'_id': u'parteien', u'count': 2},
 {u'_id': u'kabinette', u'count': 11},
 {u'_id': u'parteispenden14', u'count': 18},
 {u'_id': u'laender', u'count': 45},
 {u'_id': u'seitenwechsler', u'count': 559},
 {u'_id': u'dax', u'count': 733},
 {u'_id': u'parteispenden13', u'count': 1086},
 {u'_id': u'thinktanks', u'count': 1700},
 {u'_id': u'pr', u'count': 1866},
 {u'_id': None, u'count': 3580},
 {u'_id': u'bundestag', u'count': 11440},
 {u'_id': u'parteispenden', u'count': 16959},
 {u'_id': u'lobbyliste', u'count': 41513}]

In [121]:
# Aggregation, die das "data"-Array einer Entität zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Anschließend wird nach dem Feld data.key gruppiert etc p.p. 
pipeline = [ {"$unwind": "$data"} , {"$group": {"_id": "$data.key", "count":{"$sum": 1}}}, {"$sort": {"count":-1}}]
list(db.entities.aggregate(pipeline, cursor={}))

[{u'_id': u'source', u'count': 27828},
 {u'_id': u'titles', u'count': 14560},
 {u'_id': u'address', u'count': 12712},
 {u'_id': u'link', u'count': 5433},
 {u'_id': u'surname', u'count': 3445},
 {u'_id': u'names', u'count': 3441},
 {u'_id': u'photo', u'count': 2433},
 {u'_id': u'description', u'count': 2384},
 {u'_id': u'members', u'count': 2118},
 {u'_id': u'organisations', u'count': 953},
 {u'_id': u'topic', u'count': 684},
 {u'_id': u'bundesland', u'count': 629},
 {u'_id': u'btcertuid', u'count': 608},
 {u'_id': u'wahlkreis', u'count': 524},
 {u'_id': u'verified', u'count': 449},
 {u'_id': u'displayname', u'count': 402},
 {u'_id': u'url', u'count': 206},
 {u'_id': u'landesliste', u'count': 194},
 {u'_id': u'finance', u'count': 151},
 {u'_id': u'partei', u'count': 130},
 {u'_id': u'staff', u'count': 126},
 {u'_id': u'title', u'count': 55},
 {u'_id': u'legalform', u'count': 28},
 {u'_id': u'www', u'count': 9},
 {u'_id': u'begin', u'count': 5},
 {u'_id': u'end', u'count': 4},
 {u'_id': 

### Hilfsfunktion, die das Abfragen eines Wertes im data Array der Entitäten vereinfacht

In [ ]:
def get_cursor_given_key_value(key):
    assert type(key) == str
    pipeline = [ {"$unwind": "$data"} , {"$match": { "data.key": key}}, {"$limit":10}]
    return db.entities.aggregate(pipeline, cursor={})